In [3]:
import pdb
import matplotlib.pyplot as plt
import sys
import numpy as np
import pickle
import copy
import os

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Activation, ZeroPadding2D, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras import backend as K
from keras.utils import to_categorical


class GAN():
    def __init__(self,dataset_name='mnist',load_model_name=''):
        
        optimizer = Adam(0.0002, 0.5)
        if (load_model_name == ''):
            X_train,Y_train = self.load_gan_data(dataset_name)

            # default parameters for mnist 
            self.img_rows = X_train.shape[1]
            self.img_cols = X_train.shape[2]
            self.img_channels = X_train.shape[3]
            self.img_shape = (self.img_rows, self.img_cols, self.img_channels)
            self.z_dim = 32
            self.label_dim = Y_train.shape[1]
            self.iter_count = 0
            self.dataset_name = dataset_name
            self.model_file = 'models/'+self.dataset_name+'_cgan_model.pickle'#

            # Build and compile the discriminator
            self.discriminator = self.build_discriminator()
            # START INSERT CODE HERE
            self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
            # END INSERT CODE HERE

            # Build the generator
            self.generator = self.build_generator()

        else:
            #load gan class and models (generator, discriminator and stacked model)
            self.load_gan_model(load_model_name)

        # Create the stacked model
        #first, create the random vector z in the latent space
        z = Input(shape=(self.z_dim,))
        label = Input(shape=(self.label_dim,))
        #create generated (fake) image
        img = self.generator([z,label])

        #indicate that for the stacked model, the weights are not trained
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and gives a probability of whether it is a true or
        #false image
        p_true = self.discriminator([img,label])

        # The combined model  (stacked generator and discriminator)
        # In this model, we train the generator only
        self.stacked_gen_disc = Model([z,label], p_true)

        # loss
        # START INSERT CODE HERE
        generator_loss = K.mean(K.log(1. - p_true))
        # END INSERT CODE HERE
        self.stacked_gen_disc.add_loss(generator_loss)
        self.stacked_gen_disc.compile(optimizer=optimizer)

    def build_generator(self):

        z_rand = Input(shape=(self.z_dim,))
        label  = Input(shape=(self.label_dim,))
        inputs = Concatenate()([z_rand, label])

        # START INSERT CODE HERE
        z = Dense(256)(inputs)
        z = LeakyReLU(alpha = 0.2)(z)
        z = Dense(512)(z)
        z = LeakyReLU(alpha = 0.2)(z)
        z = Dense(784)(z)
        z = Activation('tanh')(z)
        output_img = Reshape(self.img_shape)(z)
        # END INSERT CODE HERE
        
        model_generator = Model(inputs=[z_rand, label], outputs = [output_img])
        model_generator.summary()

        return model_generator

    def build_discriminator(self):

        input_img = Input(shape=self.img_shape)
        label  = Input(shape=(self.label_dim,))

        # START INSERT CODE HERE
        input_img = Input(shape=self.img_shape)
        x = Flatten()(input_img)
        x = Dense(512)(x)
        x = LeakyReLU(alpha = 0.2)(x)
        x = Dense(256)(x)
        x = LeakyReLU(alpha = 0.2)(x)
        x = Dense(1)(x)
        p_true = Activation('sigmoid')(x)
        # END INSERT CODE HERE
        
        model_discriminator = Model(inputs=[input_img, label], outputs = p_true)
        model_discriminator.summary()

        return model_discriminator

    def load_gan_data(self,dataset_name):
        # Load the dataset
        if(dataset_name == 'mnist'):
            (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
        elif(dataset_name == 'cifar'):
            from keras.datasets import cifar10
            (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
        else:
            print('Error, unknown database')

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        #add a channel dimension, if need be (for mnist data)
        if(X_train.ndim ==3):
            X_train = np.expand_dims(X_train, axis=3)

        #modify label format
        y_train = to_categorical(Y_train)
        return X_train,y_train

    def save_gan_model(self, model_name):

        #save the GAN class instance
        gan_temp = GAN(self.dataset_name,'')
        gan_temp.generator = self.generator
        gan_temp.discriminator = self.discriminator
        gan_temp.stacked_gen_disc = []
        gan_temp.iter_count = self.iter_count
        with open(model_name+'_gan_class.pickle','wb') as file_class:
            pickle.dump(gan_temp,file_class,-1)

    def load_gan_model(self, model_name):

        #load GAN class instance
        gan_temp = pickle.load(open(model_name+'_gan_class.pickle',"rb",-1))
        #copy parameters
        self.img_rows = gan_temp.img_rows 
        self.img_cols = gan_temp.img_cols 
        self.img_channels = gan_temp.img_channels 
        self.img_shape = gan_temp.img_shape
        self.z_dim = gan_temp.z_dim
        self.iter_count = gan_temp.iter_count
        self.model_file = gan_temp.model_file
        self.dataset_name = gan_temp.dataset_name

        #copy models
        self.generator = gan_temp.generator
        self.discriminator = gan_temp.discriminator

    def train(self, epochs, batch_size=128, sample_interval=50):
        
        k=1 #number of internal loops

        #load dataset
        X_train,Y_train = self.load_gan_data(self.dataset_name)

        # labels which indicate true or false examples
        d_output_true = np.ones((batch_size, 1))
        d_output_false = np.zeros((batch_size, 1))

        first_iter =self.iter_count

        for epoch in range(first_iter,epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Train the discriminator
            for i in range(0,k):
                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                labels = Y_train[idx]
                z_random = np.random.normal(0, 1, (batch_size, self.z_dim))

                # Generate a batch of new (fake) images
                gen_imgs = self.generator.predict( [z_random,labels])

                # BEGIN INSERT CODE HERE
                d_loss_real = self.discriminator.train_on_batch([imgs, labels], d_output_true)
                d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], d_output_false)
                # END INSERT CODE HERE
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            # ---------------------
            #  Train Generator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            labels = Y_train[idx]

            z_random = np.random.normal(0, 1, (batch_size, self.z_dim))

            # Generate a batch of new (fake) images
            gen_imgs = self.generator.predict([z_random,labels])

            # Generator training : try to make generated images be classified as true by the discriminator
            g_loss = self.stacked_gen_disc.train_on_batch([z_random, labels],None)

            # increase epoch counter
            self.iter_count = self.iter_count+1
            # Plot the losses
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # Save some random generated images and the models at every sample_interval iterations
            if (epoch % sample_interval == 0):
                #save errors
                self.sample_images('images/'+self.dataset_name+'_conditional'+'_sample_%06d.png' % epoch)
                self.save_gan_model(self.model_file)

    def sample_images(self, image_filename, rand_seed=30):
        np.random.seed(rand_seed)

        r, c = 5, 5
        z_random = np.random.normal(0, 1, (r * c, self.z_dim))
        labels = np.random.randint(self.label_dim, size=(r*c))
        labels = to_categorical(labels)
        gen_imgs = self.generator.predict([z_random,labels])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                #black and white images
                if(gen_imgs.shape[3] == 1):
                    axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                elif(gen_imgs.shape[3] == 3):   #colour images
                    axs[i,j].imshow(gen_imgs[cnt, :,:])
                else:
                    print('Error, unsupported channel size. Dude, I don''t know what you want me to do.\
                            I can''t handle this data. You''ve made me very sad ...')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(image_filename)
        plt.close()


if __name__ == '__main__':

    #create the output image and model directories
    if (os.path.isdir('images')==0):
        os.mkdir('images')
    if (os.path.isdir('models')==0):
        os.mkdir('models')

    #choose dataset
    dataset_name ='mnist'

    #create GAN model
    model_file = ''#'models/'+dataset_name+'_cgan_model.pickle'#
    gan = GAN(dataset_name,model_file)#,
    is_training = 1

    if (is_training ==1):
        gan.train(epochs=14001, batch_size=64, sample_interval=500)
    else:
        gan.sample_images('images/test_images.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_122 (InputLayer)       (None, 28, 28, 1)         0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_103 (Dense)            (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_69 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_104 (Dense)            (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_70 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_105 (Dense)            (None, 1)                 257       
__________

11 [D loss: 0.335592, acc.: 86.72%] [G loss: -0.469213]
12 [D loss: 0.304584, acc.: 96.09%] [G loss: -0.436429]
13 [D loss: 0.292808, acc.: 100.00%] [G loss: -0.414757]
14 [D loss: 0.291568, acc.: 100.00%] [G loss: -0.404630]
15 [D loss: 0.282520, acc.: 100.00%] [G loss: -0.394754]
16 [D loss: 0.282405, acc.: 100.00%] [G loss: -0.390288]
17 [D loss: 0.273385, acc.: 100.00%] [G loss: -0.369405]
18 [D loss: 0.271362, acc.: 99.22%] [G loss: -0.363599]
19 [D loss: 0.269443, acc.: 100.00%] [G loss: -0.368606]
20 [D loss: 0.278683, acc.: 99.22%] [G loss: -0.372657]
21 [D loss: 0.311145, acc.: 92.19%] [G loss: -0.366064]
22 [D loss: 0.278215, acc.: 96.09%] [G loss: -0.320629]
23 [D loss: 0.316514, acc.: 96.09%] [G loss: -0.403276]
24 [D loss: 0.309703, acc.: 89.06%] [G loss: -0.372562]
25 [D loss: 0.304031, acc.: 97.66%] [G loss: -0.343807]
26 [D loss: 0.399129, acc.: 75.00%] [G loss: -0.424261]
27 [D loss: 0.370289, acc.: 69.53%] [G loss: -0.381958]
28 [D loss: 0.358121, acc.: 82.81%] [G los

160 [D loss: 0.409490, acc.: 96.09%] [G loss: -0.269014]
161 [D loss: 0.372263, acc.: 97.66%] [G loss: -0.259240]
162 [D loss: 0.373465, acc.: 99.22%] [G loss: -0.241131]
163 [D loss: 0.370959, acc.: 96.88%] [G loss: -0.261840]
164 [D loss: 0.329779, acc.: 100.00%] [G loss: -0.224079]
165 [D loss: 0.334785, acc.: 100.00%] [G loss: -0.229018]
166 [D loss: 0.356263, acc.: 96.09%] [G loss: -0.233672]
167 [D loss: 0.434134, acc.: 88.28%] [G loss: -0.271769]
168 [D loss: 0.413586, acc.: 89.06%] [G loss: -0.257408]
169 [D loss: 0.476149, acc.: 80.47%] [G loss: -0.284953]
170 [D loss: 0.538605, acc.: 64.84%] [G loss: -0.309428]
171 [D loss: 0.499508, acc.: 72.66%] [G loss: -0.283539]
172 [D loss: 0.702964, acc.: 46.88%] [G loss: -0.400185]
173 [D loss: 0.551772, acc.: 53.91%] [G loss: -0.352451]
174 [D loss: 0.601094, acc.: 55.47%] [G loss: -0.404328]
175 [D loss: 0.553814, acc.: 52.34%] [G loss: -0.364711]
176 [D loss: 0.605421, acc.: 51.56%] [G loss: -0.416862]
177 [D loss: 0.601236, acc.: 

307 [D loss: 0.521827, acc.: 72.66%] [G loss: -0.336480]
308 [D loss: 0.474769, acc.: 83.59%] [G loss: -0.301890]
309 [D loss: 0.448693, acc.: 86.72%] [G loss: -0.317551]
310 [D loss: 0.490161, acc.: 82.03%] [G loss: -0.314591]
311 [D loss: 0.457273, acc.: 82.03%] [G loss: -0.307177]
312 [D loss: 0.521961, acc.: 75.78%] [G loss: -0.332059]
313 [D loss: 0.532309, acc.: 72.66%] [G loss: -0.328994]
314 [D loss: 0.485462, acc.: 82.81%] [G loss: -0.333472]
315 [D loss: 0.638555, acc.: 60.16%] [G loss: -0.330952]
316 [D loss: 0.455237, acc.: 85.16%] [G loss: -0.303902]
317 [D loss: 0.636247, acc.: 56.25%] [G loss: -0.440577]
318 [D loss: 0.493893, acc.: 71.09%] [G loss: -0.357258]
319 [D loss: 0.495715, acc.: 79.69%] [G loss: -0.335943]
320 [D loss: 0.610298, acc.: 63.28%] [G loss: -0.382275]
321 [D loss: 0.611837, acc.: 60.94%] [G loss: -0.432893]
322 [D loss: 0.633788, acc.: 55.47%] [G loss: -0.450173]
323 [D loss: 0.603874, acc.: 59.38%] [G loss: -0.396092]
324 [D loss: 0.535474, acc.: 71

455 [D loss: 0.537680, acc.: 74.22%] [G loss: -0.378130]
456 [D loss: 0.630272, acc.: 67.97%] [G loss: -0.443327]
457 [D loss: 0.616948, acc.: 61.72%] [G loss: -0.420163]
458 [D loss: 0.555798, acc.: 69.53%] [G loss: -0.470922]
459 [D loss: 0.535796, acc.: 73.44%] [G loss: -0.384158]
460 [D loss: 0.562542, acc.: 70.31%] [G loss: -0.396083]
461 [D loss: 0.464199, acc.: 84.38%] [G loss: -0.385201]
462 [D loss: 0.539207, acc.: 76.56%] [G loss: -0.446491]
463 [D loss: 0.541347, acc.: 71.88%] [G loss: -0.388036]
464 [D loss: 0.494931, acc.: 81.25%] [G loss: -0.361767]
465 [D loss: 0.540462, acc.: 75.00%] [G loss: -0.346399]
466 [D loss: 0.513260, acc.: 78.12%] [G loss: -0.410861]
467 [D loss: 0.478722, acc.: 84.38%] [G loss: -0.362533]
468 [D loss: 0.432509, acc.: 86.72%] [G loss: -0.337721]
469 [D loss: 0.463511, acc.: 81.25%] [G loss: -0.317596]
470 [D loss: 0.559252, acc.: 72.66%] [G loss: -0.355152]
471 [D loss: 0.530823, acc.: 77.34%] [G loss: -0.363496]
472 [D loss: 0.531178, acc.: 75

...

12068 [D loss: 0.381455, acc.: 83.59%] [G loss: -0.470699]
12069 [D loss: 0.405393, acc.: 82.03%] [G loss: -0.390897]
12070 [D loss: 0.445964, acc.: 80.47%] [G loss: -0.435646]
12071 [D loss: 0.470196, acc.: 77.34%] [G loss: -0.493058]
12072 [D loss: 0.425750, acc.: 78.12%] [G loss: -0.335228]
12073 [D loss: 0.498632, acc.: 73.44%] [G loss: -0.402739]
12074 [D loss: 0.400731, acc.: 83.59%] [G loss: -0.395473]
12075 [D loss: 0.422248, acc.: 80.47%] [G loss: -0.335563]
12076 [D loss: 0.383460, acc.: 86.72%] [G loss: -0.373928]
12077 [D loss: 0.382609, acc.: 84.38%] [G loss: -0.342425]
12078 [D loss: 0.505349, acc.: 71.88%] [G loss: -0.355470]
12079 [D loss: 0.418016, acc.: 78.91%] [G loss: -0.459600]
12080 [D loss: 0.399421, acc.: 82.03%] [G loss: -0.435400]
12081 [D loss: 0.434222, acc.: 82.03%] [G loss: -0.335690]
12082 [D loss: 0.399082, acc.: 85.16%] [G loss: -0.382548]
12083 [D loss: 0.421804, acc.: 79.69%] [G loss: -0.438301]
12084 [D loss: 0.420866, acc.: 83.59%] [G loss: -0.38013

12208 [D loss: 0.432360, acc.: 78.12%] [G loss: -0.477302]
12209 [D loss: 0.447598, acc.: 78.91%] [G loss: -0.397154]
12210 [D loss: 0.358810, acc.: 83.59%] [G loss: -0.427512]
12211 [D loss: 0.431926, acc.: 82.03%] [G loss: -0.505451]
12212 [D loss: 0.411984, acc.: 82.81%] [G loss: -0.487170]
12213 [D loss: 0.428900, acc.: 81.25%] [G loss: -0.424775]
12214 [D loss: 0.441981, acc.: 78.12%] [G loss: -0.445368]
12215 [D loss: 0.409881, acc.: 83.59%] [G loss: -0.377457]
12216 [D loss: 0.413780, acc.: 79.69%] [G loss: -0.424047]
12217 [D loss: 0.399098, acc.: 82.81%] [G loss: -0.477835]
12218 [D loss: 0.464598, acc.: 83.59%] [G loss: -0.530089]
12219 [D loss: 0.415943, acc.: 80.47%] [G loss: -0.371287]
12220 [D loss: 0.440677, acc.: 81.25%] [G loss: -0.487606]
12221 [D loss: 0.379068, acc.: 82.03%] [G loss: -0.527530]
12222 [D loss: 0.402263, acc.: 82.81%] [G loss: -0.492410]
12223 [D loss: 0.428108, acc.: 82.81%] [G loss: -0.409604]
12224 [D loss: 0.400612, acc.: 85.16%] [G loss: -0.42453

12354 [D loss: 0.415191, acc.: 82.81%] [G loss: -0.385336]
12355 [D loss: 0.439232, acc.: 79.69%] [G loss: -0.412938]
12356 [D loss: 0.411300, acc.: 82.03%] [G loss: -0.436313]
12357 [D loss: 0.373550, acc.: 85.94%] [G loss: -0.507160]
12358 [D loss: 0.414318, acc.: 79.69%] [G loss: -0.401868]
12359 [D loss: 0.345398, acc.: 89.06%] [G loss: -0.530956]
12360 [D loss: 0.429718, acc.: 85.16%] [G loss: -0.519425]
12361 [D loss: 0.524674, acc.: 75.00%] [G loss: -0.442332]
12362 [D loss: 0.447855, acc.: 75.78%] [G loss: -0.409248]
12363 [D loss: 0.407425, acc.: 85.94%] [G loss: -0.418628]
12364 [D loss: 0.346880, acc.: 86.72%] [G loss: -0.458349]
12365 [D loss: 0.438982, acc.: 78.12%] [G loss: -0.519973]
12366 [D loss: 0.381610, acc.: 83.59%] [G loss: -0.595632]
12367 [D loss: 0.349318, acc.: 85.16%] [G loss: -0.387751]
12368 [D loss: 0.452351, acc.: 78.12%] [G loss: -0.429833]
12369 [D loss: 0.412538, acc.: 80.47%] [G loss: -0.329949]
12370 [D loss: 0.438161, acc.: 79.69%] [G loss: -0.40830

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_304 (InputLayer)       (None, 28, 28, 1)         0         
_________________________________________________________________
flatten_44 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_259 (Dense)            (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_173 (LeakyReLU)  (None, 512)               0         
_________________________________________________________________
dense_260 (Dense)            (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_174 (LeakyReLU)  (None, 256)               0         
_________________________________________________________________
dense_261 (Dense)            (None, 1)                 257       
__________

ResourceExhaustedError: OOM when allocating tensor with shape[512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_2/Adam/mul_6}} = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Adam_17/beta_1/read, training_2/Adam/Variable_1/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_34/mul/_881}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_640_loss_34/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
